In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import scipy
import scipy.stats
from scipy.stats import moment, norm, logistic, f, chi2
from jupyterthemes import jtplot
jtplot.style()

## Основные задания
1) По методу Монте-Карло вычислить приближенное значение определенного интеграла. Параметр числа итераций n выбрать большим 1000. Сравнить полученное значение либо с точным значением (если его получится вычислить), либо с приближенным, полученным в каком-либо математическом пакете (например, в mathematica).

## Параметры
(вариант 4)

а) $\int_0^{\pi} (x \mathrm{cos}(x))^2 dx$

б) $\int_{-\infty}^{\infty} \frac{dx}{(x^2+x+1)^2} $

в) $\iint\limits_{x^2+y^2<1} \, \mathrm{ln}\left(\frac{1}{\sqrt{x^2+y^2}}\right) dx dy $

## Дополнительные задания
1) Решить систему линейных алгебраических уравнений $x=Ax+f$ методом Монте-Карло. Сравнить с решением данного уравнения, полученным произвольным численным методом или решением в произвольном математическом пакете. В качестве матрицы $A$ взять матрицу $P$ из своего варианта лабораторной работы номер 2 и все ее элементы умножить на $0.9$. В качестве вектора $f$ выбрать вектор $\pi$ из той же лабораторной работы. Если система получается несовместной или имеет не одно решение, то разрешается изменить матрицу А, домножив некоторые ее элементы на $-1$.